In [ ]:
-!pip install lightgbm
!pip install xgboost

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import make_scorer, f1_score
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [2]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [3]:
train_data = pd.read_parquet('/content/drive/MyDrive/OrgaDeDatos/train_checkpoint2.parq')

In [4]:
not_label_df = train_data.loc[train_data["label"]==0, :]
label_df = train_data.loc[train_data["label"]==1, :]
not_label_ammount = round(len(label_df) * 100 / 7) # Regla de 3 simple
random_not_label_df = not_label_df.sample(n=not_label_ammount - len(label_df), random_state=73)
del train_data
train_data = pd.concat([label_df, random_not_label_df], axis=0)
del not_label_df, label_df

In [5]:
X_train = train_data.drop("label", axis=1)
# train_attacker_ips = train_data.loc[:, ["attacker_ip_enum"]]
y_train = train_data.loc[:, "label"]
del train_data

In [6]:
val_data = pd.read_parquet('/content/drive/MyDrive/OrgaDeDatos/val_checkpoint2.parq')

In [7]:
X_val = val_data.drop("label", axis=1)
# val_attacker_ips = val_data.loc[:, ["attacker_ip_enum"]]
y_val = val_data.loc[:, "label"]
del val_data

# RandomForest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from joblib import dump, load

random_tree = RandomForestClassifier()
# X_val.drop(["attack_hour", "attack_day"], axis=1, inplace=True)
# X_train.drop(["attack_hour", "attack_day"], axis=1, inplace=True)

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4, 10],
    'min_samples_split': [2, 5, 10, 15],
    'n_estimators': [200, 600, 1000]
              }
f1_scorer = make_scorer(f1_score)
random_search = RandomizedSearchCV(
    random_tree, param_distributions=param_grid, cv=3, n_iter=3, n_jobs=-1, random_state=37, verbose=3, scoring=f1_scorer)
random_search.fit(X_train, y_train)
# grid_search = GridSearchCV(estimator=random_tree, param_grid=param_grid, scoring=f1_scorer, cv=3)  # cv es la validación cruzada
# grid_search.fit(X_res, y_res)
best_params = random_search.best_params_
print("Mejores parámetros:", best_params)

# best_model = RandomForestClassifier(**best_params)
# best_model.fit(X_train, y_train)

# dump(best_model, 'best_model.joblib')
# dump(best_model, "/content/drive/MyDrive/OrgaDeDatos/3rd_randomTree.joblib")

In [ ]:
# pred = random_search.predict(X_val)
# train_pred = random_search.predict(X_train)

In [8]:
params =  {
    'n_estimators': 200,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'auto',
    'max_depth': 100,
    'bootstrap': False
    } # Estos fueron los hiper-parametros obtenidos

rfc = RandomForestClassifier(**params)

In [9]:
rfc.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(bootstrap=False, max_depth=100, max_features='auto',
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200)

In [ ]:
pred = rfc.predict(X_val)
train_pred = rfc.predict(X_train)

In [ ]:
train_score = f1_score(y_train.reset_index(drop=True), train_pred)
score = f1_score(y_val.reset_index(drop=True), pred)
print(f"Score para train: {train_score}")
print(f"Score para val: {score}")

In [ ]:
test_data = pd.read_parquet('/content/drive/MyDrive/aa-OrgaDeDatos/TP2/datasets/test_checkpoint2.parq')

In [ ]:
test_data.drop(["attack_hour", "attack_day"], axis=1, inplace=True)

In [ ]:
test_pred = rfc.predict(test_data)

In [ ]:
test_data = pd.read_parquet('/content/drive/MyDrive/aa-OrgaDeDatos/TP2/datasets/test_checkpoint1.parq')

In [ ]:
test_data["label"] = test_pred

In [ ]:
a = test_data.groupby("attacker_ip_enum").agg({"label":"sum", "attacker_ip_enum": "count"})

In [ ]:
b = a.label / a.attacker_ip_enum

In [ ]:
# m = b.mean()
c = b.apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
d = c.reset_index()
d.columns = ["attacker_ip_enum", "label"]
d["attacker_ip_enum"] = d.loc[:, "attacker_ip_enum"].astype("int32")
d["label"] = d.loc[:, "label"].astype("int8")

In [ ]:
d.to_csv("/content/drive/MyDrive/aa-OrgaDeDatos/TP2/random_search_no_datetime.csv", index=False)